In [14]:
import pandas as pd
df_train = pd.read_json (r'train.json')

In [2]:
c =pd.DataFrame(df_train.groupby('cuisine')['id'].nunique())
c=c.sort_values(by=['id'], ascending=False)
print(c) #Select top 3 recipies for the model

                id
cuisine           
italian       7838
mexican       6438
southern_us   4320
indian        3003
chinese       2673
french        2646
cajun_creole  1546
thai          1539
japanese      1423
greek         1175
spanish        989
korean         830
vietnamese     825
moroccan       821
british        804
filipino       755
irish          667
jamaican       526
russian        489
brazilian      467


### Create a new dataframe with only the 3 cuisines

In [3]:
crit = ((df_train['cuisine'] == 'italian') | 
                (df_train['cuisine']  == 'mexican') | 
                (df_train['cuisine']  == 'southern_us') )
df_new=df_train[crit]  

### Find the top 5 ingredients occuring in these cuisines and rate them 1 to 5, with 5 being the highest occuring(frequent)

In [4]:
l=[]
for i in df_new[df_new['cuisine']  == 'southern_us'].ingredients[0:]:
    l.append(i)

import numpy as np 
l=np.concatenate([np.array(j) for j in l])
print(l)

counts,values = pd.Series(l).value_counts().values, pd.Series(l).value_counts().index
df_results = pd.DataFrame(list(zip(values,counts)),columns=['cuisine', 'southern_us'])
# df_results.head(10)
df_results=df_results[1:6]
r=[5,4,3,2,1]
df_results.southern_us=r
# df_results_s=df_results.T
df_results_s=df_results
df_results_s.head()

['plain flour' 'ground pepper' 'salt' ... 'boiling water' 'milk' 'salt']


,cuisine,southern_us
1,butter,5
2,all-purpose flour,4
3,sugar,3
4,large eggs,2
5,baking powder,1


In [5]:
l=[]
for i in df_new[df_new['cuisine']  == 'mexican'].ingredients[0:]:
    l.append(i)

import numpy as np 
l=np.concatenate([np.array(j) for j in l])
print(l)

counts,values = pd.Series(l).value_counts().values, pd.Series(l).value_counts().index
df_results = pd.DataFrame(list(zip(values,counts)),columns=['cuisine', 'mexican'])
# df_results.head(10)
df_results=df_results[1:6]
r=[5,4,3,2,1]
df_results.mexican=r
# df_results_m=df_results.T
df_results_m=df_results
df_results_m.head()

['olive oil' 'purple onion' 'fresh pineapple' ... 'roma tomatoes' 'celery'
 'dried oregano']


,cuisine,mexican
1,onions,5
2,ground cumin,4
3,garlic,3
4,olive oil,2
5,chili powder,1


In [7]:
l=[]
for i in df_new[df_new['cuisine']  == 'italian'].ingredients[0:]:
    l.append(i)

import numpy as np 
l=np.concatenate([np.array(j) for j in l])
print(l)

counts,values = pd.Series(l).value_counts().values, pd.Series(l).value_counts().index
df_results = pd.DataFrame(list(zip(values,counts)),columns=['cuisine', 'italian'])
# print(df_results.head(10))
df_results=df_results[1:6]
r=[5,4,3,2,1]
df_results.italian=r
# df_results_i=df_results.T
df_results_i=df_results
df_results_i.head()

['sugar' 'pistachio nuts' 'white almond bark' ... 'pitted black olives'
 'Kraft Grated Parmesan Cheese' 'red pepper']


,cuisine,italian
1,olive oil,5
2,garlic cloves,4
3,grated parmesan cheese,3
4,garlic,2
5,ground black pepper,1


In [8]:
f=pd.concat([df_results_i, df_results_m,df_results_s],  ignore_index=True).reset_index(drop=True)#.T   
f=f.T
print(f)

                    0              1                       2       3   \
cuisine      olive oil  garlic cloves  grated parmesan cheese  garlic   
italian              5              4                       3       2   
mexican            NaN            NaN                     NaN     NaN   
southern_us        NaN            NaN                     NaN     NaN   

                              4       5             6       7          8   \
cuisine      ground black pepper  onions  ground cumin  garlic  olive oil   
italian                        1     NaN           NaN     NaN        NaN   
mexican                      NaN       5             4       3          2   
southern_us                  NaN     NaN           NaN     NaN        NaN   

                       9       10                 11     12          13  \
cuisine      chili powder  butter  all-purpose flour  sugar  large eggs   
italian               NaN     NaN                NaN    NaN         NaN   
mexican                

In [9]:
df2 = pd.DataFrame(f)      #convert the first row in header
df2.columns = df2.iloc[0]
df2 = df2[1:]
final=df2.groupby(df2.columns, axis=1).sum()

In [15]:
final.reset_index(level=0,inplace=True)
final.rename(columns={'index':'cuisine'},inplace=True)
final

cuisine,cuisine,cuisine,all-purpose flour,baking powder,butter,chili powder,garlic,garlic cloves,grated parmesan cheese,ground black pepper,ground cumin,large eggs,olive oil,onions,sugar
0,0,italian,0.0,0.0,0.0,0.0,2.0,4.0,3.0,1.0,0.0,0.0,5.0,0.0,0.0
1,1,mexican,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,4.0,0.0,2.0,5.0,0.0
2,2,southern_us,4.0,1.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,3.0


#### convert cuisines to categories

In [28]:
pd.Categorical(final.cuisine)

['cuisine', 'cuisine']
Categories (1, object): ['cuisine']

In [35]:
final

cuisine,cuisine,all-purpose flour,baking powder,butter,chili powder,garlic,garlic cloves,grated parmesan cheese,ground black pepper,ground cumin,large eggs,olive oil,onions,sugar
0,0,0.0,0.0,0.0,0.0,2.0,4.0,3.0,1.0,0.0,0.0,5.0,0.0,0.0
1,1,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,4.0,0.0,2.0,5.0,0.0
2,2,4.0,1.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,3.0


In [34]:
final = final.loc[:,~final.columns.duplicated()]

In [36]:
from sklearn.naive_bayes import GaussianNB

target=final.cuisine
features=final.iloc[:,1:]

model = GaussianNB()

# Train the model using the training sets
model.fit(features,target)

GaussianNB()

In [37]:
#Predict Output

#Rate the favorite ingredients on the scale of 1-5 with 5 being the best. Put "0" for the ones which you don't want to rate.

a=5    #(all-purpose flour)
b=4    #(baking powder)
c=2    #(butter)
d=5    #(chili powder)
e=0    #(garlic)
f=0    #(garlic cloves)
g=0    #(grated parmesan cheese)
h=0    #(ground black pepper)
i=0    #(ground cumin)
j=0    #(large eggs)
k=5    #(olive oil)
l=0    #(onions)
m=0    #(sugar)


predicted= model.predict([[a,b,c,d,e,f,g,h,i,j,k,l,m]]) 
print("Predicted Value:", predicted)

Predicted Value: [2]


The predcitited cusine type is - southern_us